In [1]:
%load_ext autoreload
%autoreload 2
%autosave 0

from etl import kaggle, rivm
from contagion import *

Autosave disabled


In [14]:
cprob, ctime = 0.02, 5

confirmed = kaggle().xs('US').xs('New York')['confirmed']
history = confirmed.shift(-ctime, freq='D').dropna() / cprob
history.tail()

Date
2020-03-21    1893850.0
2020-03-22    2243800.0
2020-03-23    2620500.0
2020-03-24    2982400.0
2020-03-25    3333150.0
Freq: D, Name: confirmed, dtype: float64

In [15]:
from numpy import linspace

window = linspace(1, 0.1, 14)
window /= window.sum()
window = [0, *window]
window

[0,
 0.12987012987012986,
 0.12087912087912088,
 0.11188811188811189,
 0.1028971028971029,
 0.0939060939060939,
 0.08491508491508491,
 0.0759240759240759,
 0.06693306693306693,
 0.05794205794205794,
 0.04895104895104895,
 0.03996003996003996,
 0.030969030969030958,
 0.021978021978021973,
 0.012987012987012988]

In [16]:
self = Contagion(window, beta=3, total=19.5e6)
self

Contagion(0.0, 0.12987012987012986, 0.12087912087912088, 0.11188811188811189, 0.1028971028971029, 0.0939060939060939, 0.08491508491508491, 0.0759240759240759, 0.06693306693306693, 0.05794205794205794, 0.04895104895104895, 0.03996003996003996, 0.030969030969030958, 0.021978021978021973, 0.012987012987012988, beta=3.0, dprob=0.1, dtime=15, qprob=0.5, qtime=15, total=19500000, vprob=0.0)

In [17]:
steps = 7

params = self.params
window = self.window

beta = params['beta']
total = params['total']
vprob = params['vprob']

deltas = ( y - x for x, y in zip(history, history[1:]) )
deltas = deque(deltas, steps)

exposed = history[-1]
vaccinated = total * vprob
susceptible = total - (vaccinated + exposed)
for _ in range(steps):

    print(*map(int, (exposed, susceptible, vaccinated)))
    
    delta = beta * sum(w * x for w, x in zip(reversed(window), deltas))
    delta *= susceptible / total
    deltas.append(delta)
    susceptible -= delta
    exposed += delta

    

3333150 16166850 0
3573290 15926709 0
3796472 15703527 0
4000399 15499600 0
4182541 15317458 0
4343343 15156656 0
4483397 15016602 0
